# The GAP (Gendered Ambiguous Pronouns) dataset

The dataset is designed for coreference resolution tasks, specifically for resolving ambiguous pronouns to their correct antecedents. It contains English sentences with ambiguous pronouns and corresponding entities. The primary goal is to develop models that can correctly identify whether a given pronoun refers to "A," "B," or neither.

Here's a brief summary of the structure of the GAP dataset:

1. **Columns:**
   - **ID:** A unique identifier for each example.
   - **Text:** The text of the sentence containing the ambiguous pronoun.
   - **Pronoun:** The ambiguous pronoun in the sentence.
   - **Pronoun-offset:** The offset (position) of the pronoun in the sentence.
   - **A, B:** The candidate entities to which the pronoun may refer.
   - **A-offset, B-offset:** The offsets of entities A and B in the sentence.
   - **A-coref, B-coref:** Binary labels indicating whether the pronoun refers to entities A or B.

2. **Labels:**
   - **A-coref, B-coref:** These binary labels are used for training the model. A label of 1 indicates that the pronoun refers to the corresponding entity, and 0 indicates it does not.

3. **Task:**
   - The task associated with this dataset is to build a model that, given a sentence with an ambiguous pronoun, predicts whether the pronoun refers to entity A, entity B, or neither.

Here is a snippet of what the data might look like:

```plaintext
ID, Text, Pronoun, Pronoun-offset, A, A-offset, B, B-offset, A-coref, B-coref
example1, "John met Susan in the park. He said she had a dog.", he, 35, John, 0, Susan, 16, True, False
example2, "Alice and Bob went to the store. They bought groceries.", they, 35, Alice, 0, Bob, 11, True, False
```

In this example, the model needs to predict whether "he" refers to John or Susan and whether "they" refers to Alice or Bob.



# The `RankNetModel`

RankNetModel is a simple neural network model designed for pairwise ranking tasks, such as the task of ranking ambiguous pronoun candidates in coreference resolution. Let's break down the components and discuss their relevance to the task:

```python
class RankNetModel(nn.Module):
    def __init__(self, input_dim):
        super(RankNetModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
```

1. **Initialization (`__init__` method):**
   - `input_dim`: This parameter represents the dimensionality of the input features. In the case of coreference resolution, it could be the dimensionality of the feature vectors representing pairs of mentions (e.g., TF-IDF vectors or embeddings).

   - `nn.Linear(input_dim, 64)`: This is the first fully connected (linear) layer. It takes the input features and maps them to a 64-dimensional intermediate representation.

   - `nn.ReLU()`: The Rectified Linear Unit (ReLU) activation function is applied element-wise after the first linear layer. ReLU introduces non-linearity to the model, allowing it to learn complex relationships in the data.

   - `nn.Linear(64, 1)`: The second linear layer reduces the 64-dimensional representation to a single output. In the context of RankNet, this output is interpreted as the predicted ranking score for a pair of mentions.

```python
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
```

2. **Forward Pass (`forward` method):**
   - `x`: This represents the input features, such as the TF-IDF vectors or embeddings for pairs of mentions.

   - `self.relu(self.fc1(x))`: The input features pass through the first linear layer, followed by the ReLU activation function. This introduces non-linearity to the model's transformations.

   - `self.fc2(x)`: The output of the first layer is then passed through the second linear layer, producing a single-dimensional output. In the context of pairwise ranking, this output can be interpreted as the predicted ranking score for the pair of mentions.

   - `return x`: The final output is returned, representing the model's predicted ranking score for the input pair of mentions.

**Relevance to the Task:**
   - The model is designed for pairwise ranking, which is suitable for tasks where the goal is to rank pairs of items. In coreference resolution, this can be used to rank pairs of candidate antecedents for an ambiguous pronoun.

   - The model architecture with two linear layers and a ReLU activation allows the network to capture complex relationships and patterns in the input data.

   - The single-dimensional output from the model can be used to compare and rank pairs of mentions, aiding in the decision of whether a pronoun refers to one entity over another.

   - The choice of activation functions and the architecture is common in neural network models for ranking tasks, providing a balance between expressiveness and simplicity.

In summary, the `RankNetModel` is a neural network architecture tailored for the task of pairwise ranking, making it relevant for scenarios like coreference resolution where the goal is to rank potential antecedents for ambiguous pronouns.

# Preprocessing

Certainly! Let's break down the preprocessing steps:

1. **Loading the Dataset:**
   ```python
   url = "https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv"
   gap_data = pd.read_csv(url, sep='\t')
   ```
   - The dataset is loaded from the provided URL using `pd.read_csv`. The `sep='\t'` parameter indicates that the data is tab-separated.

2. **Creating Pairs and Labels:**
   ```python
   pairs = []
   labels = []

   for index, row in gap_data.iterrows():
       mention1 = row["Text"]
       mention2 = row["Pronoun"]

       # Assign label based on whether the pronoun refers to the same entity (1) or not (0)
       label = 1 if row["A-coref"] or row["B-coref"] else 0

       pairs.append({"mention1": mention1, "mention2": mention2})
       labels.append(label)
   ```
   - For each row in the dataset, two mentions (`mention1` and `mention2`) are extracted from the columns "Text" and "Pronoun."

   - The label is assigned based on whether the pronoun refers to entity A or B (1) or neither (0).

   - Pairs of mentions and their corresponding labels are stored in the `pairs` and `labels` lists.

3. **Feature Engineering with TF-IDF:**
   ```python
   vectorizer = TfidfVectorizer()
   features = vectorizer.fit_transform([pair["mention1"] + " " + pair["mention2"] for pair in pairs])
   ```
   - A `TfidfVectorizer` is used to convert pairs of mentions into TF-IDF (Term Frequency-Inverse Document Frequency) vectors.

   - The TF-IDF vectors are computed based on the concatenation of `mention1` and `mention2` for each pair.

4. **Converting to PyTorch Tensors:**
   ```python
   X = torch.tensor(features.toarray(), dtype=torch.float32)
   y = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)
   ```
   - The TF-IDF vectors (`features`) are converted to a PyTorch tensor (`X`) with a data type of `torch.float32`.

   - The labels (`labels`) are also converted to a PyTorch tensor (`y`) with the same data type. Additionally, `unsqueeze(1)` is used to convert the 1D tensor into a column vector, as RankNet expects labels in this format.

The resulting `X` and `y` tensors can be used for training and evaluating the RankNet model on the pairwise ranking task.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# Define the RankNet model
class RankNetModel(nn.Module):
    def __init__(self, input_dim):
        super(RankNetModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load GAP dataset from the URL
url = "https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv"
gap_data = pd.read_csv(url, sep='\t')

# Preprocess data: create pairs of mentions and labels
pairs = []
labels = []

for index, row in gap_data.iterrows():
    mention1 = row["Text"]
    mention2 = row["Pronoun"]

    # Assign label based on whether the pronoun refers to the same entity (1) or not (0)
    label = 1 if row["A-coref"] or row["B-coref"] else 0

    pairs.append({"mention1": mention1, "mention2": mention2})
    labels.append(label)

# Feature engineering: use TF-IDF vectors as features
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform([pair["mention1"] + " " + pair["mention2"] for pair in pairs])

# Convert features and labels to PyTorch tensors
X = torch.tensor(features.toarray(), dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # RankNet expects labels in column vector form

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RankNet model
input_dim = X_train.shape[1]
model = RankNetModel(input_dim)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy with Logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 500

for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)

    # Create pairwise labels
    pairwise_labels = torch.sign(y_train.view(-1, 1) - y_train.view(1, -1))

    # Reshape the model's output to match the pairwise labels and convert to float
    outputs = outputs.view(-1, 1).expand(-1, len(y_train)).float()

    # Compute the loss using the pairwise labels
    loss = criterion(outputs, pairwise_labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate the model on the test set
with torch.no_grad():
    test_outputs = model(X_test)
    test_predictions = torch.argsort(test_outputs.view(-1))

    precision = torch.sum((test_predictions == torch.arange(len(y_test))).float()) / len(y_test)
    print("Precision:", precision.item())


Precision: 0.0024999999441206455
